In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
import pyheif

In [ ]:
# 1. BLIP 모델과 Processor 불러오기
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [ ]:
# HEIC 이미지를 RGB로 변환하는 함수
def load_heic_image(image_path):
    heif_file = pyheif.read(image_path)  # .HEIC 파일 읽기
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
    )
    return image.convert("RGB")  # RGB로 변환하여 반환

In [ ]:
# # 이미지 경로 입력받아 데이터프레임 생성 함수
def create_food_dataframe(image_paths):
    data = []

    for image_path in image_paths:
        # 이미지 로드 및 전처리
        image = load_heic_image(image_path)
        inputs = processor(image, return_tensors="pt")

        # BLIP 모델로 음식 이름 예측
        out = model.generate(**inputs)
        food_name = processor.decode(out[0], skip_special_tokens=True).split(",")[0]  # 첫 번째 단어를 음식 이름으로 추출

        # 데이터 추가
        data.append({"음식이름": food_name})

    # 데이터프레임 생성
    df = pd.DataFrame(data)
    return df

In [ ]:
image_paths = [
    '/content/drive/MyDrive/Colab Notebooks/A4/images/img1.HEIC',
    '/content/drive/MyDrive/Colab Notebooks/A4/images/img2.HEIC',
    '/content/drive/MyDrive/Colab Notebooks/A4/images/img3.HEIC',
    '/content/drive/MyDrive/Colab Notebooks/A4/images/img4.HEIC',
    '/content/drive/MyDrive/Colab Notebooks/A4/images/img5.HEIC'
]

In [ ]:
# 데이터프레임 생성 및 출력
food_df = create_food_dataframe(image_paths)
food_df

,음식이름
0,a bowl of food on a table
1,a plate of food with meat and vegetables
2,a basket of fried chicken
3,a plate of food on a table
4,a plate of pasta with clams and clams


In [ ]:
food_df.to_csv('/content/drive/MyDrive/Colab Notebooks/A4/food_df.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/A4/food_df.csv')
food_names = df['음식이름'].tolist()
food_names

['a bowl of food on a table',
 'a plate of food with meat and vegetables',
 'a basket of fried chicken',
 'a plate of food on a table',
 'a plate of pasta with clams and clams']

In [ ]:
import openai

# OpenAI API 키 설정
openai.api_key = ""


In [ ]:
def start_conversation(food_names):
    # 음식 목록을 포함한 사용자에게 대화 시작
    food_message = f"The user has recently eaten the following foods: {', '.join(food_names)}. Based on this, help them choose their next meal."

    # 카테고리별 질문 설정
    categories = [
        "매운거 또는 안매운거",
        "따뜻한거 또는 시원한거",
        "한식, 양식, 중식, 일식 중 선택",
        "빵, 밥, 면, 단백질 중 선택",
        "간식, 파티, 혼밥, 야식 중 선택"
    ]

    # 대화 메시지 준비
    messages = [
        {"role": "system", "content": "You are a helpful assistant who understands food preferences."},
        {"role": "user", "content": food_message}
    ]

    # 카테고리별로 질문을 던지고 사용자 응답 받기
    user_responses = {}  # 사용자 응답 저장
    for category in categories:
        # 각 카테고리별로 질문을 추가
        category_message = f"Please choose one from the following options: {category}"
        print(category_message)  # 사용자에게 질문 출력

        user_input = input(f"{category} (Your answer): ")  # 사용자로부터 입력 받기
        user_responses[category] = user_input  # 사용자의 응답 저장

        # 대화 내용에 사용자의 입력 추가
        user_message = f"The user selected: {user_input}"
        messages.append({"role": "user", "content": user_message})

    # 마지막에 추천된 음식과 레시피 제공 요청
    final_message = "Based on your preferences, please recommend a meal and provide the recipe."
    messages.append({"role": "user", "content": final_message})

    # 최종적으로 추천된 메뉴와 레시피 받기
    final_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    # 최종 응답 내용 반환
    return final_response['choices'][0]['message']['content']


In [ ]:
# 대화 실행
response_content = start_conversation(food_names)

# 응답 내용 출력
print("\n--- 최종 추천 ---")
print(response_content)

Please choose one from the following options: 매운거 또는 안매운거
매운거 또는 안매운거 (Your answer): 매운거
Please choose one from the following options: 따뜻한거 또는 시원한거
따뜻한거 또는 시원한거 (Your answer): 따뜻한거
Please choose one from the following options: 한식, 양식, 중식, 일식 중 선택
한식, 양식, 중식, 일식 중 선택 (Your answer): 한식
Please choose one from the following options: 빵, 밥, 면, 단백질 중 선택
빵, 밥, 면, 단백질 중 선택 (Your answer): 밥
Please choose one from the following options: 간식, 파티, 혼밥, 야식 중 선택
간식, 파티, 혼밥, 야식 중 선택 (Your answer): 간식

--- 최종 추천 ---
Considering your preference for a warm Korean meal, I recommend making a delicious and comforting bowl of Bibimbap. Bibimbap is a popular Korean dish that consists of steamed rice topped with sautéed vegetables, a protein of your choice (such as beef or tofu), a fried egg, and a flavorful gochujang (Korean chili paste) sauce. Here's a simple Bibimbap recipe for you to try:

Ingredients:
- Cooked white rice
- Assorted vegetables (suggestions: carrots, spinach, bean sprouts, zucchini, mushrooms